In [1]:
import math
from pickletools import optimize
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from PIL import Image
import os
import random
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
from tensorflow.keras.layers import (Dense, Dropout, Flatten, GlobalAveragePooling2D, Input)
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

TRAIN_DATA_DIR = 'B:/23k_Verchovskiy/ii/CatsDogs/CatsDogs/train'
VALIDATION_DATA_DIR = 'B:/23k_Verchovskiy/ii/CatsDogs/CatsDogs/test'
TRAIN_SAMPLES = 1000
VALIDATION_SAMPLES = 400
NUM_CLASSES = 2
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 2

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, zoom_range=0.2)    
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_datagen.flow_from_directory(TRAIN_DATA_DIR, target_size = (IMG_WIDTH, IMG_HEIGHT), batch_size=BATCH_SIZE, shuffle=True, seed=12345, class_mode='categorical')
validation_generator = val_datagen.flow_from_directory(VALIDATION_DATA_DIR, target_size=(IMG_WIDTH, IMG_HEIGHT), batch_size=BATCH_SIZE, shuffle=False, seed=12345, class_mode='categorical')

# СОЗДАНИЕ МОДЕЛИ НЕЙРОННОЙ СЕТИ #

def model_maker():
    base_model = MobileNet(include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    for layer in base_model.layers[:]:
        layer.trainable=False
    input=Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    custom_model = base_model(input)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Dense(10, activation = 'relu')(custom_model)
    custom_model = Dropout(0.25)(custom_model)
    predictions = Dense(NUM_CLASSES, activation = 'softmax')(custom_model)
    return Model(inputs=input, outputs=predictions)

# КОМПИЛИРУЕМ И ОБУЧАЕМ МОДЕЛЬ #
model = model_maker()
model.compile(loss = 'categorical_crossentropy', optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00001), metrics = ['acc'])
num_steps = math.ceil(float(TRAIN_SAMPLES) / BATCH_SIZE)
history = model.fit(train_generator, steps_per_epoch = num_steps, epochs = 10, validation_data = validation_generator, validation_steps = num_steps)

# СТРОИМ ГРАФИК #
loss_function = history.history['loss']
val_loss_function = history.history['val_loss']
epochs = range(1, len(loss_function) + 1)
plt.title('Losses')
plt.plot(epochs, loss_function, color = 'blue', label = 'Потери в обучающей выборке')
plt.plot(epochs, val_loss_function, color = 'red', label = 'Потери в валидационной выборке')
plt.xlabel('Epoch')
plt.ylabel('Loss value')
plt.legend()
plt.show()

import os
import random
categ = ['cat', 'dog']
files = []
for root, dirs, filenames in os.walk('B:/23k_Verchovskiy/ii/CatsDogs/CatsDogs/test/cats'):
    for filename in filenames:
        files.append(os.path.join(root, filename))
for root, dirs, filenames in os.walk('B:/23k_Verchovskiy/ii/CatsDogs/CatsDogs/test/dogs'):
    for filename in filenames:
        files.append(os.path.join(root, filename))
for f in random.sample(files, 10):
    img_path = f
    img = image.load_img(img_path, target_size = (224, 224))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis = 0)
    preprocessed_img = preprocess_input(expanded_img_array)
    prediction = model.predict(preprocessed_img)
    plt.title(categ[np.argmax(prediction)])
    plt.imshow(img)
    plt.show()

ModuleNotFoundError: No module named 'tensorflow.keras'